<a href="https://colab.research.google.com/github/talgalper/Honours-2021/blob/main/nextstrain_data_treatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install ncbi-blast+
!apt-get install exonerate
!apt-get install dssp
!pip install biostructmap

In [ ]:
import pandas as pd
from collections import defaultdict
import biostructmap

In [14]:
def nextstrain_data_processing(filename, seq_len, gene_id):
    ns_data = pd.read_csv(filename, sep='\t')

    s_gene_data = ns_data[ns_data.gene == gene_id]

    pos = s_gene_data.position

    entropy = s_gene_data.entropy

    # Default dict will return zero if not in original entropy data.
    pos_to_entropy = defaultdict(int, zip(pos, entropy))

    len_of_ref_seq = seq_len

    # Create a list that includes zeros.
    list_of_values = [pos_to_entropy[i+1] for i in range(len_of_ref_seq)]

    with open('process_diversity_data.txt', 'w') as f:
        for entropy in list_of_values:
            f.write(f"{entropy:0.3f}\n")

In [15]:
nextstrain_data_processing('nextstrain_ncov_global_diversity.tsv', 1275, 'S')

In [21]:
def biostructmap_analysis(pdb_structure, gene_name, chains, output_filename):
    # Initialise structure object
    structure = biostructmap.Structure(pdb_structure, gene_name)

    data = {(chains): ['procecssed_diversity_data.txt']}

    # Note: Use method='default' for mapping an ordered list of values (must contain values for each residue in the sequence.)
    results = structure.map(data, method='default', ref=None, radius=15)

    results.write_data_to_pdb_b_factor(fileobj=output_filename)

In [ ]:
biostructmap_analysis('6xm0.pdb', 'sars_cov_2_spike', ('a,b,c'), 'biostructmap_global_diversity.pdb')